# Decoding Neuraly Relevant Information from the Shape of the Hemodynamic Response
A walk-through the three neuromimaging data sets acquired during my PhD.

This documentation is not prospective (constructed as the work unfolds) but retrospective (at a stage close to classic publication). Yes, shame on me.

I am therefore constructing this documentation backward, from the highly processed data of final figures back toward the raw data. Note that I may never get to the raw data due to time constrains, so many projects, this imperfect current system of scholarly publishing and you know, life.

## Brain Decoding of Visual Stimulus Orientation With Cyclical Stimulus Paradigm

Stimuli were presented at one of two orthogonal orientations (+/- 45 degrees) in alternating 6-sec ON and 6-sec OFF blocs. This produced sinusoidal BOLD signal variations in V1 cortex.

In [49]:
clear all
dataDir = 'C:\Users\sebas\OneDrive - McGill University\McGill\work\projects\170210_HRdecoding\C_processing';
fileList = {'02jp_sess1' '03sk_sess1' '04sp_sess1' '05bm_sess1' '06sb_sess1' '07bj_sess1';...
            '02jp_sess2' '03sk_sess2' '04sp_sess2' '05bm_sess2' '06sb_sess2' '07bj_sess2'}';
fileList


fileList =

  6x2 cell array

    {'02jp_sess1'}    {'02jp_sess2'}
    {'03sk_sess1'}    {'03sk_sess2'}
    {'04sp_sess1'}    {'04sp_sess2'}
    {'05bm_sess1'}    {'05bm_sess2'}
    {'06sb_sess1'}    {'06sb_sess2'}
    {'07bj_sess1'}    {'07bj_sess2'}




In [50]:
% Load complex numbers representing the sinusoidal response
for sessInd = 1:size(fileList,2);
    for subjInd = 1:size(fileList,1);
        load(fullfile(dataDir,[fileList{subjInd,sessInd} '.mat']),'d')
        ori1{subjInd,sessInd} = d.xData(d.label==1,:);
        ori2{subjInd,sessInd} = d.xData(d.label==2,:);
        plaid{subjInd,sessInd} = d.normData;
    end
end
size(ori1)
size(ori1{subjInd,sessInd})


ans =

     6     2


ans =

           6        2939




In [51]:
% Average voxels
for sessInd = 1:size(fileList,2);
    for subjInd = 1:size(fileList,1);
        ori1{subjInd,sessInd} = mean(ori1{subjInd,sessInd},2);
        ori2{subjInd,sessInd} = mean(ori2{subjInd,sessInd},2);
        plaid{subjInd,sessInd} = mean(plaid{subjInd,sessInd},2);
    end
end
size(ori1)
size(ori1{subjInd,sessInd})


ans =

     6     2


ans =

     6     1




In [52]:
for sessInd = 1:size(fileList,2);
    for subjInd = 1:size(fileList,1);
        %average runs
        ori1_m(subjInd,sessInd) = mean(ori1{subjInd,sessInd},1);
        ori2_m(subjInd,sessInd) = mean(ori2{subjInd,sessInd},1);
        plaid_m(subjInd,sessInd) = mean(plaid{subjInd,sessInd},1);
    end
end
size(ori1_m)


ans =

     6     2




In [53]:
% Average ori
ori_m = mean(cat(3,ori1_m,ori2_m),3);
size(ori_m)


ans =

     6     2




In [54]:
% Average sessions
ori1_m = mean(ori1_m,2);
ori2_m = mean(ori2_m,2);
ori_m = mean(ori_m,2);
plaid_m = mean(plaid_m,2);
size(ori_m)


ans =

     6     1




In [55]:
% Stats
% amplitude
disp('amplitude')
[H,P,CI,STATS] = ttest(abs(ori_m),abs(plaid_m))
%% delay
disp('delay')
[H,P,CI,STATS] = ttest(angle(ori_m),angle(plaid_m))
[P,H,STATS] = signrank(angle(ori_m),angle(plaid_m))
[P,F] = circ_htest(angle(ori_m),angle(plaid_m))

amplitude

H =

     0


P =

    0.3742


CI =

   -0.0721
    0.1603


STATS = 

  struct with fields:

    tstat: 0.9752
       df: 5
       sd: 0.1107

delay

H =

     1


P =

    0.0310


CI =

    0.0076
    0.1044


STATS = 

  struct with fields:

    tstat: 2.9738
       df: 5
       sd: 0.0461


P =

    0.0625


H =

  logical

   0


STATS = 

  struct with fields:

    signedrank: 20


P =

    0.1046


F =

    4.1832


